In [2]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time


# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [3]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [19]:
def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = 0.0
    x_ex += sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1])
    x_ex += sum(y_in[(g, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = 0.0 
    x_gp += sum(y_el[(g, 2)] for g in range(num_el))
    x_gp += sum(y_in[(g, 1)] for g in range(num_in))
    
    ell_ex = sum(a_ex[p] * (x_ex ** p) for p in range(5))
    ell_gp = sum(a_gp[p] * (x_gp ** p) for p in range(5))
    
    obj_E = tau * sum(y_el[(g, 1)] for g in range(num_el)) \
                + sum(y_el[(g, k)] * v_E_array[g] for g in range(num_el) for k in [0, 1]) * ell_ex \
                + sum(y_el[(g, 2)] * v_E_array[g] for g in range(num_el)) * ell_gp
    obj_I = tau * sum(y_in[(g, 0)] for g in range(num_in)) \
                + sum(y_in[(g, 0)] * v_I_array[g] for g in range(num_in)) * ell_ex \
                + sum(y_in[(g, 1)] * v_I_array[g] for g in range(num_in)) * ell_gp  
    obj_R = tau * sum(y_el[(g, 1)] for g in range(num_el)) \
                + tau * sum(y_in[(g, 0)] for g in range(num_in))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
    print()
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print("welfare:", welfare)
    print()

    return welfare

def latency_total(tau, v_I_array, v_E_array, y_el, y_in, \
                  a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                  a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = 0.0
    x_ex += sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1])
    x_ex += sum(y_in[(g, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = 0.0 
    x_gp += sum(y_el[(g, 2)] for g in range(num_el))
    x_gp += sum(y_in[(g, 1)] for g in range(num_in))
    
    ell_ex = sum(a_ex[p] * (x_ex ** p) for p in range(5))
    ell_gp = sum(a_gp[p] * (x_gp ** p) for p in range(5))
        
    latency_val = x_ex * ell_ex + x_gp * ell_gp
    
    return latency_val


In [20]:
def proj_tau_B_11(tau, B, tau_max = 1.0, B_max = 1.0):
    
    tau_feas = cp.Variable(1)
    B_feas = cp.Variable(1)
    
    func = (tau_feas - tau)**2 + (B_feas - B)**2

    objective = cp.Minimize(func)
    
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= tau_feas]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas.value, 0.0), decimals=4)


# Chinmay's Algorithm:

In [21]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [22]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_CBCP_direct(tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")

    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for k in [0, 1, 2]:
            y_el[(g, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for k in [0, 1]:
            y_in[(g, k)] = cp.Variable(1)
            
    x_ex = cp.Variable(1)
    x_gp = cp.Variable(1)
    
    # Objective:
    func = 0.0
    
    func += 1/5 * a_ex[4] * cp.power(x_ex, 5)
    func += 1/4 * a_ex[3] * cp.power(x_ex, 4)
    func += 1/3 * a_ex[2] * cp.power(x_ex, 3)
    func += 1/2 * a_ex[1] * cp.power(x_ex, 2)
    func += a_ex[0] * x_ex
    func += 1/5 * a_gp[4] * cp.power(x_gp, 5)
    func += 1/4 * a_gp[3] * cp.power(x_gp, 4)
    func += 1/3 * a_gp[2] * cp.power(x_gp, 3)
    func += 1/2 * a_gp[1] * cp.power(x_gp, 2)
    func += a_gp[0] * x_gp
    
    func += sum(y_el[(g, 1)] / v_E_array[g] for g in range(num_el)) * tau
    func += sum(y_in[(g, 0)] / v_I_array[g] for g in range(num_in)) * tau

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, k)] >= 0.0 for g in range(num_el) for k in [0, 1, 2]]
    constraints += [y_in[(g, k)] >= 0.0 for g in range(num_in) for k in [0, 1]]
    
    constraints += [sum(y_el[(g, k)] for k in [0, 1, 2] ) == 1.0 for g in range(num_el)]
    constraints += [sum(y_in[(g, k)] for k in [0, 1] ) == 1.0 for g in range(num_in)]
    
    constraints += [x_ex == sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1]) \
                               + sum(y_in[(g, 0)] for g in range(num_in))]
    constraints += [x_gp == sum(y_el[(g, 2)] for g in range(num_el)) \
                               + sum(y_in[(g, 1)] for g in range(num_in))]
    
    constraints += [y_el[(g, 0)] * tau <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for k in [0, 1, 2]:
            y_el_values[(g, k)] = y_el[(g, k)].value[0]
            
    y_in_values = {}
    for g in range(num_in):
        for k in [0, 1]:
            y_in_values[(g, k)] = y_in[(g, k)].value[0]

    return y_el_values, y_in_values


In [17]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 2.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
tau_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros(num_iters)
tau_perturbed = np.zeros(num_iters)
B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(2)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[i] = tau[i] + delta[i] * w_i[0]
    B_perturbed[i] = B[i] + delta[i] * w_i[1]
    
#     if tau_perturbed[i] < B_perturbed[i] or tau_perturbed[i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[i], B_perturbed[i] = proj_tau_B_11(tau_perturbed[i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[i]:", tau[i])
    print("B[i]:", B[i])
    print("tau_perturbed[i]:", tau_perturbed[i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(tau = tau[i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

    print()
    print("y_el_values:", y_el_values)
    print("y_in_values:", y_in_values)
    print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(tau = tau_perturbed[i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau_perturbed[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    
    welfare_list.append(welfare)
    
#     def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I, v_E, y_el, y_in, \
#                 a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
#                 a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0]))
    
    print("welfare:", welfare)
    print("welfare_perturbed:", welfare_perturbed)
#     print("tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare):\n", tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare))
    
    tau[i+1] = tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare)
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[1] * (welfare_perturbed - welfare)
    tau[i+1], B[i+1] = proj_tau_B_11(tau[i+1], B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = tau[i-diffs_num_cols : i-1] - tau[i-diffs_num_cols+1 : i]
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[0:10]:", tau[0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
w_i: [ 0.31534246 -0.94897794]
tau[i]: 4.199999999999999
B[i]: 1.7999999999999998
tau_perturbed[i]: 4.8689
B_perturbed[i]: 0.0

y_el_values: {(0, 0): 0.42857141782322644, (0, 1): 1.4966304195951352e-08, (0, 2): 0.5714285672104693, (1, 0): 0.4285714208275619, (1, 1): 2.5652277191312578e-08, (1, 2): 0.5714285535201609, (2, 0): 0.42857142307206897, (2, 1): 4.049808211004091e-08, (2, 2): 0.5714285364298489}
y_in_values: {(0, 0): 5.932492508353704e-08, (0, 1): 0.9999999406750749, (1, 0): 0.6980772671871015, (1, 1): 0.3019227328128985, (2, 0): 0.9999998033974922, (2, 1): 1.966025078186952e-07}


obj_E: 186.90781765817633
obj_R: 7.131924286309964
obj_I: 297.2505717301261
welfare: 669.6398221892168


obj_E: 191.9656355365948
obj_R: 14.496892084500088
obj_I: 297.5365731197895
welfare: 678.5684657760792

welfare: 669.6398221892168
welfare_perturbed: 678.5684657760792

Iter: 1
w_i: [0.23673744 0.97157366]
tau[i]: 1.0
B[i]: 1.0
tau_perturbed[i]: 2.0777
B_perturbed[i]: 2.0777

y_el_values:


obj_E: 186.30145077014595
obj_R: 5.774464186518237e-07
obj_I: 291.59772995233465
welfare: 664.2006313771373


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003

welfare: 664.2006313771373
welfare_perturbed: 664.2000000000003
tau_diffs: [ 1.      0.      0.     -0.0038]
B_diffs: [ 0.9986  0.      0.     -0.0038]

Iter: 10
w_i: [0.23932798 0.97093878]
tau[i]: 0.0034
B[i]: 0.0031
tau_perturbed[i]: 0.7081
B_perturbed[i]: 0.7081

y_el_values: {(0, 0): 0.9117609138263905, (0, 1): 2.0470826472829273e-06, (0, 2): 0.08823703909096225, (1, 0): 0.9117603172495231, (1, 1): 3.893681729455478e-06, (1, 2): 0.08823578906874743, (2, 0): 0.9117599234572273, (2, 1): 5.344931378131745e-06, (2, 2): 0.08823473161139453}
y_in_values: {(0, 0): 4.712690877317627e-06, (0, 1): 0.9999952873091227, (1, 0): 4.259744236234653e-06, (1, 1): 0.9999957402557638, (2, 0): 0.26469182341877795, (2, 1): 0.735308176581222}


obj_E: 186.29861638668686
obj_R: 0.0009000210772687977
obj_I: 


y_el_values: {(0, 0): 0.9999529118061002, (0, 1): 1.701441420420835e-06, (0, 2): 4.538675247933721e-05, (1, 0): 0.9999635928947477, (1, 1): 2.6880376263061696e-06, (1, 2): 3.3719067626016766e-05, (2, 0): 0.9999721253363125, (2, 1): 3.177758698837212e-06, (2, 2): 2.469690498863994e-05}
y_in_values: {(0, 0): 5.415440818778983e-06, (0, 1): 0.9999945845591812, (1, 0): 7.598301726252643e-06, (1, 1): 0.9999924016982737, (2, 0): 8.72184593758707e-06, (2, 1): 0.9999912781540624}


obj_E: 186.27961707031602
obj_R: 1.3772328327245969e-06
obj_I: 291.63190955952075
welfare: 664.1911434247063


obj_E: 186.76552481320078
obj_R: 1.2040480699982963
obj_I: 292.0775243664523
welfare: 665.3677643788542

welfare: 664.1911434247063
welfare_perturbed: 665.3677643788542
tau_diffs: [-0.0059 -0.0035  0.0007 -0.0082]
B_diffs: [-0.007   0.0009  0.0004 -0.0099]

Iter: 20
w_i: [-0.79914214 -0.60114211]
tau[i]: 0.276
B[i]: 0.276
tau_perturbed[i]: 0.0
B_perturbed[i]: 0.0

y_el_values: {(0, 0): 0.9999473826829752, (

tau_diffs: [-0.0092  0.0056 -0.3435 -0.0008]
B_diffs: [ 0.0063  0.0022 -0.3927  0.0013]

Iter: 29
w_i: [0.67110688 0.74136061]
tau[i]: 0.8754
B[i]: 0.7745
tau_perturbed[i]: 1.4837
B_perturbed[i]: 1.4465

y_el_values: {(0, 0): 0.8847383536748631, (0, 1): 2.701142920447097e-08, (0, 2): 0.11526161931370771, (1, 0): 0.884738351013626, (1, 1): 4.25212573901268e-08, (1, 2): 0.11526160646511664, (2, 0): 0.884738349777084, (2, 1): 6.657497689244174e-08, (2, 2): 0.11526158364793912}
y_in_values: {(0, 0): 5.505112798953604e-08, (0, 1): 0.999999944948872, (1, 0): 2.7065029550854547e-08, (1, 1): 0.9999999729349704, (2, 0): 0.3428987677998896, (2, 1): 0.6571012322001104}


obj_E: 185.74941740629734
obj_R: 0.30017377236515624
obj_I: 292.7246019207546
welfare: 664.1634019788762


obj_E: 185.14640642142064
obj_R: 0.10432802969580406
obj_I: 293.5106336758523
welfare: 663.7825809127544

welfare: 664.1634019788762
welfare_perturbed: 663.7825809127544
tau_diffs: [ 0.0056 -0.3435 -0.0008 -0.1386]
B_diffs: 


obj_E: 186.26303451384157
obj_R: 4.483528329290843e-06
obj_I: 291.65787864433685
welfare: 664.1839467753143


obj_E: 186.2658185452906
obj_R: 3.77103371153812e-06
obj_I: 291.65351813903874
welfare: 664.1851544754132

welfare: 664.1839467753143
welfare_perturbed: 664.1851544754132
tau_diffs: [0.     0.     0.     0.0006]
B_diffs: [ 0.      0.0033 -0.0033  0.0015]

Iter: 39
w_i: [-0.96372846  0.26688471]
tau[i]: 1.0
B[i]: 0.9996
tau_perturbed[i]: 0.7059
B_perturbed[i]: 0.7059

y_el_values: {(0, 0): 0.9995994194090855, (0, 1): 1.4352000876627358e-07, (0, 2): 0.000400437070905757, (1, 0): 0.9995994470477516, (1, 1): 1.8564186095776153e-07, (1, 2): 0.00040036731038743195, (2, 0): 0.9995994691256385, (2, 1): 2.2401030497980028e-07, (2, 2): 0.00040030686405655787}
y_in_values: {(0, 0): 2.82527701345181e-07, (0, 1): 0.9999997174722987, (1, 0): 5.701303619209952e-07, (1, 1): 0.9999994298696381, (2, 0): 1.4066183405825328e-06, (2, 1): 0.9999985933816594}


obj_E: 186.00262263391235
obj_R: 2.812

...

tau_diffs: [-0.0054 -0.0066 -0.0178  0.1384]
B_diffs: [-0.0003 -0.0103  0.0013  0.1179]

Iter: 88
w_i: [ 0.66559808 -0.74631039]
tau[i]: 0.0
B[i]: 0.0
tau_perturbed[i]: 0.4597
B_perturbed[i]: 0.0

y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.83261442404282
obj_R: 1.3781153765898102
obj_I: 292.1472351800096
welfare: 665.5368409527773

welfare: 664.2000000000003
welfare_perturbed: 665.5368409527773
tau_diffs: [-0.0066 -0.0178  0.1384  0.    ]
B_diffs: [-0.0103  0.0013  0.1179  0.    ]

Iter: 89
w_i: [ 0.99057677 -0.13695861]
t

tau_diffs: [ 0.0002  0.0014 -0.0014  0.0014]
B_diffs: [ 0.0006  0.001  -0.0014  0.0014]

Iter: 97
w_i: [-0.18635431  0.98248261]
tau[i]: 0.0077
B[i]: 0.0077
tau_perturbed[i]: 0.2761
B_perturbed[i]: 0.2761

y_el_values: {(0, 0): 0.9999277209290018, (0, 1): 4.003564394339243e-06, (0, 2): 6.827550660382098e-05, (1, 0): 0.9999289821191031, (1, 1): 5.755568847703196e-06, (1, 2): 6.526231204921432e-05, (2, 0): 0.9999363006857069, (2, 1): 7.295020606881819e-06, (2, 2): 5.640429368618237e-05}
y_in_values: {(0, 0): 1.9266159718256404e-05, (0, 1): 0.9999807338402817, (1, 0): 4.783697167598522e-05, (1, 1): 0.999952163028324, (2, 0): 0.0001221569255440702, (2, 1): 0.9998778430744559}


obj_E: 186.2998307300188
obj_R: 1.588619423061718e-06
obj_I: 291.60026660404196
welfare: 664.1999277463557


obj_E: 186.27035805267204
obj_R: 1.9071196163814718e-06
obj_I: 291.64640746773176
welfare: 664.1871231916518

welfare: 664.1999277463557
welfare_perturbed: 664.1871231916518
tau_diffs: [ 0.0014 -0.0014  0.001


obj_E: 186.29967538494327
obj_R: 0.0006000401198893816
obj_I: 291.60108395626963
welfare: 664.2003147181322


obj_E: 186.49612199345745
obj_R: 0.5095659099183681
obj_I: 291.80296915531784
welfare: 664.6932999602491

welfare: 664.2003147181322
welfare_perturbed: 664.6932999602491
tau_diffs: [-0.0022  0.0002  0.002  -0.002 ]
B_diffs: [-0.0022  0.0002  0.002  -0.002 ]

Iter: 107
w_i: [-0.95642624 -0.29197406]
tau[i]: 0.0789
B[i]: 0.0789
tau_perturbed[i]: 0.0
B_perturbed[i]: 0.0

y_el_values: {(0, 0): 0.9999539571066673, (0, 1): 1.6983191852837365e-06, (0, 2): 4.434457414737025e-05, (1, 0): 0.9999483533678367, (1, 1): 1.9737975268124225e-06, (1, 2): 4.9672834636470046e-05, (2, 0): 0.9999488482323557, (2, 1): 2.3837533243733074e-06, (2, 2): 4.876801431987969e-05}
y_in_values: {(0, 0): 3.023677734748098e-06, (0, 1): 0.9999969763222653, (1, 0): 5.163939394425832e-06, (1, 1): 0.9999948360606056, (2, 0): 6.324711534944072e-06, (2, 1): 0.9999936752884651}


obj_E: 186.2681425389901
obj_R: 1.622


obj_E: 186.27489009484623
obj_R: 1.7653242348460083e-06
obj_I: 291.6393110685667
welfare: 664.1890909051943


obj_E: 186.3800069177653
obj_R: 1.335790011480948
obj_I: 292.68281023230827
welfare: 665.1756660655426

welfare: 664.1890909051943
welfare_perturbed: 665.1756660655426
tau_diffs: [ 1.0e-04  0.0e+00  3.2e-03 -8.1e-02]
B_diffs: [ 1.0e-04  3.0e-04  2.9e-03 -8.1e-02]

Iter: 116
w_i: [-0.57125909 -0.8207698 ]
tau[i]: 0.0
B[i]: 0.0
tau_perturbed[i]: 0.0
B_perturbed[i]: 0.0

y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.300


y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.30732304235772
obj_R: 0.017999823118352805
obj_I: 291.6065383247957
welfare: 664.2175844448875

welfare: 664.2000000000003
welfare_perturbed: 664.2175844448875
tau_diffs: [ 0.     -0.0531  0.0035  0.0536]
B_diffs: [ 0.     -0.0531  0.0035  0.0536]

Iter: 126
w_i: [ 0.93055625 -0.36614895]
tau[i]: 0.0037
B[i]: 0.0037
tau_perturbed[i]: 0.5917
B_perturbed[i]: 0.0

y_el_values: {(0, 0): 0.9999264206794425, (0, 1): 1.2889338771748804e-05, (0, 2): 6.0689981785766456e-05, (1, 0): 0.9999

In [24]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)
print()

latency_val = latency_total(tau, v_I_array, v_E_array, y_el = y_el_values, y_in = y_in_values, \
                            a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                            a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0]))
print("latency:", latency_val)

first(welfare_list): 669.6398221892168
min(welfare_list): 663.8538307552467
max(welfare_list): 669.6398221892168
argmin_tau: 1.0
argmin_B: 0.9986

latency: 486.00000001403816


## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Scratch Work:

In [3]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


x.value: [-0.  1.]
y.value: [-0.  1.]



# Scratch Work:

In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
